## popcornTime2

In [1]:
import pandas as pd 
from scipy import sparse 
from sklearn.metrics.pairwise import cosine_similarity 

In [27]:
## Data set imported is from MovieLens 
movieRatings = pd.read_csv('ratings.csv')
movie = pd.read_csv('movies.csv')
movieDF = pd.merge(movie,movieRatings).drop(['genres', 'timestamp'], axis=1)
movieDF.head()



movieId             title  userId  rating
0        1  Toy Story (1995)       1     4.0
1        1  Toy Story (1995)       5     4.0
2        1  Toy Story (1995)       7     4.5
3        1  Toy Story (1995)      15     2.5
4        1  Toy Story (1995)      17     4.5

In [28]:
## This will give us what rating each user gave to each movie 
aggregateRatings = movieDF.pivot_table(index=['userId'], columns = ['title'], values='rating')
aggregateRatings.head()

title   '71 (2014)  'Hellboy': The Seeds of Creation (2004)  \
userId                                                        
1              NaN                                      NaN   
2              NaN                                      NaN   
3              NaN                                      NaN   
4              NaN                                      NaN   
5              NaN                                      NaN   

title   'Round Midnight (1986)  'Salem's Lot (2004)  \
userId                                                
1                          NaN                  NaN   
2                          NaN                  NaN   
3                          NaN                  NaN   
4                          NaN                  NaN   
5                          NaN                  NaN   

title   'Til There Was You (1997)  'Tis the Season for Love (2015)  \
userId                                                               
1                             NaN                              NaN   
2                             NaN                              NaN   
3                             NaN                              NaN   
4                             NaN                              NaN   
5                             NaN                              NaN   

title   'burbs, The (1989)  'night Mother (1986)  (500) Days of Summer (2009)  \
userId                                                                          
1                      NaN                   NaN                          NaN   
2                      NaN                   NaN                          NaN   
3                      NaN                   NaN                          NaN   
4                      NaN                   NaN                          NaN   
5                      NaN                   NaN                          NaN   

title   *batteries not included (1987)  \
userId                                   
1                                  NaN   
2                                  NaN   
3                                  NaN   
4                                  NaN   
5                                  NaN   

title                     ...                      Zulu (2013)  [REC] (2007)  \
userId                    ...                                                  
1                         ...                              NaN           NaN   
2                         ...                              NaN           NaN   
3                         ...                              NaN           NaN   
4                         ...                              NaN           NaN   
5                         ...                              NaN           NaN   

title   [REC]² (2009)  [REC]³ 3 Génesis (2012)  \
userId                                           
1                 NaN                      NaN   
2                 NaN                      NaN   
3                 NaN                      NaN   
4                 NaN                      NaN   
5                 NaN                      NaN   

title   anohana: The Flower We Saw That Day - The Movie (2013)  \
userId                                                           
1                                                     NaN        
2                                                     NaN        
3                                                     NaN        
4                                                     NaN        
5                                                     NaN        

title   eXistenZ (1999)  xXx (2002)  xXx: State of the Union (2005)  \
userId                                                                
1                   NaN         NaN                             NaN   
2                   NaN         NaN                             NaN   
3                   NaN         NaN                             NaN   
4                   NaN         NaN                             NaN   
5                

In [29]:
## Instead of using the NaN values and creating a bias, I will remove them from consideration.
## Thereby we will only use results of movies that have at least 5 views

aggregateRatings = aggregateRatings.dropna(thresh=5,axis=1).fillna(0)
aggregateRatings.head()

title   'burbs, The (1989)  (500) Days of Summer (2009)  \
userId                                                    
1                      0.0                          0.0   
2                      0.0                          0.0   
3                      0.0                          0.0   
4                      0.0                          0.0   
5                      0.0                          0.0   

title   *batteries not included (1987)  10 Cloverfield Lane (2016)  \
userId                                                               
1                                  0.0                         0.0   
2                                  0.0                         0.0   
3                                  0.0                         0.0   
4                                  0.0                         0.0   
5                                  0.0                         0.0   

title   10 Things I Hate About You (1999)  10,000 BC (2008)  \
userId                                                        
1                                     0.0               0.0   
2                                     0.0               0.0   
3                                     0.0               0.0   
4                                     0.0               0.0   
5                                     0.0               0.0   

title   101 Dalmatians (1996)  \
userId                          
1                         0.0   
2                         0.0   
3                         0.0   
4                         0.0   
5                         0.0   

title   101 Dalmatians (One Hundred and One Dalmatians) (1961)  \
userId                                                           
1                                                     0.0        
2                                                     0.0        
3                                                     0.0        
4                                                     0.0        
5                                                     0.0        

title   102 Dalmatians (2000)  12 Angry Men (1957)          ...            \
userId                                                      ...             
1                         0.0                  0.0          ...             
2                         0.0                  0.0          ...             
3                         0.0                  0.0          ...             
4                         0.0                  5.0          ...             
5                         0.0                  0.0          ...             

title   Zero Effect (1998)  Zodiac (2007)  Zombieland (2009)  \
userId                                                         
1                      0.0            0.0                0.0   
2                      0.0            0.0                3.0   
3                      0.0            0.0                0.0   
4                      0.0            0.0                0.0   
5                      0.0            0.0                0.0   

title   Zoolander (2001)  Zootopia (2016)  [REC] (2007)  eXistenZ (1999)  \
userId                                                                     
1                    0.0              0.0           0.0              0.0   
2                    0.0              0.0           0.0              0.0   
3                    0.0              0.0           0.0              0.0   
4                    0.0              0.0           0.0              0.0   
5                    0.0              0.0           0.0              0.0   

title   xXx (2002)  xXx: State of the Union (2005)  ¡Three Amigos! (1986)  
userId                                                                     
1              0.0                             0.0                    4.0  
2              0.0                             0.0                    0.0  
3              0.0                             0.0                    0.0  
4              0.0                             0.0      

In [32]:
## We will use pearson correlation which is a statistic that measures linear correlation between two variables X and Y. 
## It has a value between +1 and −1, where 1 is total positive linear correlation, 0 is no linear correlation, and −1 is 
## total negative linear correlation

movieSimilarity = aggregateRatings.corr(method='pearson')
movieSimilarity.head(100)

title                                               'burbs, The (1989)  \
title                                                                    
'burbs, The (1989)                                            1.000000   
(500) Days of Summer (2009)                                   0.063117   
*batteries not included (1987)                                0.235908   
10 Cloverfield Lane (2016)                                   -0.023768   
10 Things I Hate About You (1999)                             0.143482   
10,000 BC (2008)                                              0.011998   
101 Dalmatians (1996)                                         0.087931   
101 Dalmatians (One Hundred and One Dalmatians)...            0.224052   
102 Dalmatians (2000)                                        -0.018608   
12 Angry Men (1957)                                           0.034223   
12 Years a Slave (2013)                                       0.009277   
127 Hours (2010)                                              0.008331   
13 Going on 30 (2004)                                         0.049700   
13 Hours (2016)                                              -0.013602   
13th Warrior, The (1999)                                      0.176071   
1408 (2007)                                                   0.131993   
15 Minutes (2001)                                             0.104500   
16 Blocks (2006)                                              0.064842   
17 Again (2009)                                               0.047271   
1984 (Nineteen Eighty-Four) (1984)                            0.184452   
2 Days in the Valley (1996)                                   0.092824   
2 Fast 2 Furious (Fast and the Furious 2, The) ...            0.084066   
20 Dates (1998)                                               0.110264   
20,000 Leagues Under the Sea (1954)                           0.166599   
200 Cigarettes (1999)                                         0.231762   
2001: A Space Odyssey (1968)                                  0.205977   
2010: The Year We Make Contact (1984)                         0.187019   
2012 (2009)                                                   0.006377   
2046 (2004)                                                  -0.016983   
21 (2008)                                                    -0.027152   
...                                                                ...   
Abraham Lincoln: Vampire Hunter (2012)                        0.181083   
Absent-Minded Professor, The (1961)                           0.480802   
Absolute Power (1997)                                         0.181400   
Abyss, The (1989)                                             0.234777   
Accepted (2006)                                               0.035891   
Accidental Tourist, The (1988)                                0.012571   
Accused, The (1988)                                           0.020688   
Ace Ventura: Pet Detective (1994)                             0.118632   
Ace Ventura: When Nature Calls (1995)                         0.138795   
Across the Universe (2007)                                    0.035167   
Adaptation (2002)                                             0.135332   
Addams Family Values (1993)                                   0.180183   
Addams Family, The (1991)                                     0.288059   
Addicted to Love (1997)                                       0.309652   
Adjustment Bureau, The (2011)                                 0.073435   
Adventureland (2009)                                          0.234500   
Adventures in Babysitting (1987)                              0.326594   
Adventures of Baron Munchausen, The (1988)                    0.149953   
Adventures of Buckaroo Banzai Across the 8th Di...            0.402125   
Adventures of Milo and Otis, The (Koneko monoga...            0.380379   
Adventures of Pinocchio, The (1996)                          -0.015158   
Adve

In [38]:
## By subtracting 2.5 from the user score (2.5 is the avg from 1 to 5) 
## We get a work around of not promoting movies that users have rated but didnt like

def find_similar_movies(name, rating):
    score = movieSimilarity[name]*(rating-2.5)
    score = score.sort_values(ascending = False)
    return score


In [45]:
## Testing for a hypothetical thriller/action lover (Me!)
## Ratings are on a scale 1 to 5 - 5 means I liked them and 1 means I did not

ali_favs = [("2012 (2009)", 4), ("10 Cloverfield Lane (2016)", 4), 
            ("2 Fast 2 Furious (Fast and the Furious 2, The) (2003)", 5),
           ("(500) Days of Summer (2009)", 1)]

recommendMovies = pd.DataFrame()

for movie, score in ali_favs:
    recommendMovies = recommendMovies.append(find_similar_movies(movie, score), ignore_index=True);

#recommendMovies.head()
recommendMovies.sum().sort_values(ascending=False)

2 Fast 2 Furious (Fast and the Furious 2, The) (2003)                                                                 2.705301
2012 (2009)                                                                                                           2.100726
Gamer (2009)                                                                                                          2.036703
Hansel & Gretel: Witch Hunters (2013)                                                                                 1.977553
Riddick (2013)                                                                                                        1.853212
10 Cloverfield Lane (2016)                                                                                            1.838311
Fast & Furious (Fast and the Furious 4, The) (2009)                                                                   1.817744
Crank (2006)                                                                                                   